# Experiments for the Energy Efficiency Master thesis in DYNAMOS

This Jupyter notebook is used for the experiments for the master thesis in DYNAMOS about energy efficiency.

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html


## Step 1: Configure the Environment Configure the Environment (has to be done once in the Jupyter Hub environment), Create Slice, Setup K8s Cluster & Configure DYNAMOS

Before running this notebook, you will need to configure your environment using the [Configure Environment](../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook. Note: this has to be done only once in the Jupyter Hub environment (unless configuration is removed/deleted of course).

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

Next, you will need to have setup the slice in FABRIC using the [Create Slice](../create_slice.ipynb) notebook.

Finally, configure the kubernetes environment using the [Configure Kubernetes](../k8s-cluster-setup/k8s_setup.ipynb) and [Configure DYNAMOS Specifically](../dynamos/DYNAMOS_setup.ipynb) notebooks.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Step 2.1: Import FABRIC API and other libraries

In [ ]:
import json
import traceback
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### Step 2.2: Configure the parameters and variables
Can be used to set the corresponding slice and other variables used for subsequent cells.

In [ ]:
slice_name = 'DYNAMOS_EnergyEfficiency'
# Nodes:
node1_name = 'k8s-control-plane'
node2_name = 'dynamos-core'
node3_name = 'vu'
node4_name = 'uva'
node5_name = 'surf'

### Step 2.3: Extend the Slice before Running Experiments
This code extends the slice's end date to two weeks (the current maximum lease time) to make sure that the slice can be used for a longer period of time.

In [ ]:
# TODO: the function is working, but it does not save in the slice information afterwards, which is likely a bug in FABRIC. So, for now just skip this part, 
# the slice is created with a 2 weeks period, which should be enough and otherwise the steps to create it can be repeated
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Print slice information for debugging purposes
    slice.show()
    # Print lease end before
    print(f"Lease End (UTC) Before        : {slice.get_lease_end()}")

    # See https://github.com/fabric-testbed/jupyter-examples/blob/main/fabric_examples/fablib_api/renew_slice/renew_slice.ipynb
    # Calculate the end date for now + x days
    end_date = (datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S %z")
    # Renew the slice with the calculated end date
    slice.renew(end_date)

    # Verify changes afterwards
    print(f"Lease End (UTC) After       : {slice.get_lease_end()}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Step 3: Execute Experiments
TODO: explain here as well how to switch at the end, such as TODO:refer to fabric/dynamos setup to upload charts and then run after loading the dynamos-configs.sh: redeploy_structurally
TODO

### Step 3.1: Prepare FABRIC Kubernetes environment
TODO: add here any things that need to be done, such as port-forward, etc.?

```sh
TODO: here port-forward on k8s-control-plane node via SSH session? Cannot do node execute, since that is not persistent likely.
```

### Step 3.2: Prepare Node for Experiments
TODO: explain this prepares the node, needs to be done once.
TODO: here load the python files onto the node and execute them
TODO: run from which node, dynamos-core or k8s-control-plane node?

In [ ]:
try:
    # Get slice by name: https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.get_slice
    slice = fablib.get_slice(name=slice_name)
    # Get the correct node to run the experiment on
    # TODO: which node, dynamos-core or k8s-control-plane node? For now I think k8s-control-plane node, since we execute all the scripts there, and dynamos-core is required for the rest.
    node = slice.get_node(name=node1_name)
    # Make sure the directory exists:
    node.execute("mkdir -p ~/experiments")
    # Upload required files for the experiments preparation (cannot do ~, this will cause No such file, need to do relative path from home of the node):
    node.upload_file(local_file_path="prepare_node.sh", remote_file_path="./experiments/prepare_node.sh")

    # Add necessary permissions and execute the script to prepare the node environment
    stdout, stderr = node.execute(f"chmod +x ./experiments/prepare_node.sh && ./experiments/prepare_node.sh")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

### Step 3.3: Execute/Run Actual Experiments
TODO: explain, here it now runs the actual experiments on the node, this can be done multiple times by changing the variables used.
TODO: add something so that I can intermediately get the resulting files to my local machine so I can upload them to GitHub.

In [ ]:
# Change these variables for different experiments, such as archetypes, implementations, etc. 
# See energy-efficiency/experiments/README.md for explanation on how to use the script and some examples. And see execute_experiments.py for options.
CURRENT_EXP_ARCHETYPE = "ComputeToData"
CURRENT_EXP_REPS = 30
CURRENT_EXP_NAME = "baseline"
try:
    # Upload required files for the experiments:
    node.upload_file(local_file_path="constants.py", remote_file_path="./experiments/constants.py")
    node.upload_file(local_file_path="execute_experiments.py", remote_file_path="./experiments/execute_experiments.py")
    
    # Run the experiment. This needs to run the python script to allow the output to be added in the notebook output, with a separate script that did not happen
    stdout, stderr = node.execute(
        (
            # Make the script executable and 
            f"chmod +x ./experiments/execute_experiments.py && "
            # Go to the corresponding location
            f"cd ~/experiments && "
            # Activate the venv
            f"source dynamos-env/bin/activate && "
            # Execute the actual experiments. Use -u to use unbuffered mode for stdout, stderr, and stdin, 
            # so that print() calls and logs from inside the python script appear live (or at least flush immediately after each action)
            f"python3 -u execute_experiments.py {CURRENT_EXP_ARCHETYPE} {CURRENT_EXP_REPS} {CURRENT_EXP_NAME}"
        )
    )

except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()